## Step 1: Data Preparation

In [3]:
# install pathology-whole-slide-data
!pip3 install git+https://github.com/DIAGNijmegen/pathology-whole-slide-data@main

  Cloning https://github.com/DIAGNijmegen/pathology-whole-slide-data (to revision main) to c:\users\luukn\appdata\local\temp\pip-req-build-eadjm1xs
  Resolved https://github.com/DIAGNijmegen/pathology-whole-slide-data to commit 1e4c6ca939c5e372a0b626739c24443f31bff505
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wholeslidedata: filename=wholeslidedata-0.1.0-py3-none-any.whl size=73203 sha256=759666999921ea24ad8267a3d05e6d5f955ce35d1af1a75739e8a05b3aa411dc
  Stored in directory: C:\Users\luukn\AppData\Local\Temp\pip-ephem-wheel-cache-r2qzk6h2\wheels\34\3a\83\6d4167be019e68b1b698457c6d25f2d6e25796f29ecd1e68d2
Successfully built wholeslidedata

   -------- ------------------------------- 1/5 [rtree]
   ---------------- ----------------------- 2/5 [dicfg]
   ------------------------ --------------- 3/5 [concurrentbuffer]
   -------------------------------- ------- 4/5 [wholeslidedata]
   --------------------------

  Running command git clone --filter=blob:none --quiet https://github.com/DIAGNijmegen/pathology-whole-slide-data 'C:\Users\luukn\AppData\Local\Temp\pip-req-build-eadjm1xs'
  DEPRECATION: Building 'wholeslidedata' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wholeslidedata'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
import os
from utils.dot2polygon import dot2polygon
import glob
import yaml

In [5]:
# make a folder for converted annotations
output_root = r'C:/Users/luukn/AIMI_MONKEY2/monkey-training/annotations_polygon'
if not(os.path.isdir(output_root)): os.mkdir (output_root) 

Convert the dot annotation to polygon bounding boxes

In [18]:
# convert dot annotations to polygon, single image
xml_path = 'C:/Users/luukn/AIMI_MONKEY2/monkey-training/annotations/xml/A_P000001.xml'
output_path = 'C:/Users/luukn/AIMI_MONKEY2/monkey-training/annotations_polygon/xml/A_P000001.xml'
lymphocyte_half_box_size = 4.5 # the size of half of the bbox around the lymphocyte dot in um
monocytes_half_box_size = 11.0 # the size of half of the bbox around the monocytes dot in um
min_spacing = 0.25
dot2polygon(xml_path, lymphocyte_half_box_size, monocytes_half_box_size, min_spacing, output_path)

In [19]:
# convert dot annotations to polygon, in a folder

annotation_dir = r'./data/annotations'
annotation_polygon_dir = r'./data/annotations_polygon'
annotation_list = glob.glob(os.path.join(annotation_dir,'*.xml'))
for xml_path in annotation_list:
    print(xml_path)
    output_path = os.path.join(annotation_polygon_dir,os.path.splitext(os.path.basename(xml_path))[0] + '_polygon' + os.path.splitext(os.path.basename(xml_path))[1])
    print(output_path)
    lymphocyte_half_box_size = 4.5 # the size of half of the bbox around the lymphocyte dot in um
    monocytes_half_box_size = 11.0 # the size of half of the bbox around the monocytes dot in um
    min_spacing = 0.25
    dot2polygon(xml_path, lymphocyte_half_box_size, monocytes_half_box_size, min_spacing, output_path)

Creating the yaml config files for training.

In [20]:
def folders_to_yml(wsi_dir: str, 
                   wsa_dir: str,
                   output_dir: str,
                   output_name: str):

    """
    Generate a yaml file to be used as WSD dataconfig from a folder of slides and a folder of annotation or mask files.
    Assumes files use the same name for both the slides and masks.
    """
    
    wsa_list = glob.glob(wsa_dir)

    yaml_dict = {'training': []}
    # yaml_dict = {'training': [], 'validation': []}
    for wsa in wsa_list:
        patient_name = os.path.basename(wsa).split(os.path.basename(wsa_dir).split('*')[1])[0] # monocytes
    #     print(patient_name)
        if os.path.isfile(os.path.join(wsi_dir,patient_name + "_PAS_CPG.tif")):
            wsi = os.path.join(wsi_dir, patient_name + "_PAS_CPG.tif")
            print('match found:    ' , patient_name)
            yaml_dict['training'].append(
                    {"wsa": {"path": str(wsa)}, "wsi": {"path": str(wsi)}})

            # # validation if needed
            # yaml_dict['validation'].append(
            #         {"wsa": {"path": str(wsa)}, "wsi": {"path": str(wsi)}})

        else:
            print('no match found:    ' , patient_name)

    # make a folder for output
    if not(os.path.isdir(output_dir)): os.mkdir (output_dir)


    with open(os.path.join(output_dir,output_name), "w") as file:
            yaml.safe_dump(yaml_dict, file)

In [21]:
wsi_dir = r'./data/images'
wsa_dir = r'./data/annotations_polygon/*_polygon.xml'
output_dir = r'./configs'
output_name = 'training_sample.yml'

folders_to_yml(wsi_dir, 
               wsa_dir,
               output_dir,
               output_name)